In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models 
import numpy as np
class SimpleCNN:
    def __init__(self, input_shape, num_classes):
        self.model=self.build_model(input_shape ,num_classes)
        
    def build_model(self,input_shape,num_classes):
        model =models.Sequential([
            layers.Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=input_shape),
            layers.MaxPooling2D(pool_size=(2,2)),

            layers.Conv2D(64,kernel_size =(3,3),activation='relu'),
            layers.MaxPooling2D(pool_size=(2,2)),

            layers.Conv2D(128,kernel_size =(3,3),activation='relu'),
            layers.MaxPooling2D(pool_size=(2,2)),
            layers.Flatten(),
            layers.Dense(256,activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(num_classes,activation='softmax')
            
            
            
        ])
        model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
        return model
    def train(self,x_train,y_train,epochs=20,batch_size=128,validation_data=None):
        self.model.fit(x_train,y_train,epochs=epochs,batch_size=batch_size,validation_data=validation_data)
    def predict(self,x):
        return self.model.predict(x)
    def evaluate(self,x_test,y_test):
        return self.model.evaluate(x_test,y_test)
    


In [ ]:
class CNNEnsemble:
    def __init__(self,models):
        self.models=models
    def predict(self,x):
        predictions =np.zeros((x.shape[0],10))
        for model in self.models:
            predictions+=model.predict(x)
        predictions/=len(self.models)
        return predictions

    def evaluate(self,x_test,y_test):
        predictions =self.predict(x_test)
        accuracy= np.mean(np.argmax(predictions,axis=1)==np.argmax(y_test,axis=1)
                         )
        return accuracy

In [ ]:
# CIFAR-10 데이터셋 사용
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# 데이터 전처리
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [ ]:
# CNN 모델 여러 개 생성
cnn1 = SimpleCNN(input_shape=(32, 32, 3), num_classes=10)
cnn2 = SimpleCNN(input_shape=(32, 32, 3), num_classes=10)
cnn3 = SimpleCNN(input_shape=(32, 32, 3), num_classes=10)

# 각 CNN 모델 훈련
cnn1.train(x_train, y_train, epochs=20, validation_data=(x_test, y_test))
cnn2.train(x_train, y_train, epochs=20, validation_data=(x_test, y_test))
cnn3.train(x_train, y_train, epochs=20, validation_data=(x_test, y_test))


In [ ]:

# CNN 앙상블
ensemble_model = CNNEnsemble([cnn1, cnn2, cnn3])

# 앙상블 평가
accuracy = ensemble_model.evaluate(x_test, y_test)
print(f'Ensemble Model Accuracy: {accuracy:.4f}')